In [1]:
import torch
import scipy.io as io
from __future__ import print_function, division
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch.nn as nn
import torch.nn.functional as F

In [2]:
headers='hw5_data_code/deap_features.mat'
dataHeaders= io.loadmat(headers)

dataFile = 'hw5_data_code/results_DEAP.mat'
#dataFile= io.loadmat(dataFile)
#fname=data['label','PID','image','tumorBorder','tumorMask'] 

In [3]:
features_frame = np.zeros(shape=(1280, 6))
print(features_frame)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [4]:
class FeaturesDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.features_frame = io.loadmat(data_file)#pd.read_csv(csv_file)
        self.features_frame = np.zeros(shape=(1280, 6))
        i = 0
        #print(dataHeaders.shape)
        #for line in dataHeaders:
        subjects = dataHeaders['features']
        # read each feature set for each video for this subject into the data array
        for subject in subjects:
            for feature in subject:
                # get mean of means for EMG features
                mean_emg = np.mean(feature['EMG_feats'], axis=0)
                mean_emg = mean_emg[0]
                #print(mean_emg)
                self.features_frame[i][0] = mean_emg
                # set GSR mean
                #print(feature['GSR_feats'])
                self.features_frame[i][1] = feature['GSR_feats'][0][3]
                # set RES mean
                self.features_frame[i][2] = feature['RES_feats'][0][0]
                # set BVP mean
                #print("BVP")
                #print(feature['BVP_feats'])
                self.features_frame[i][3] = feature['BVP_feats'][0][0]
                self.features_frame[i]
                #print(feature['feedback']['felt_arousal'])
                #print(feature['feedback']['felt_arousal'])
                if feature['feedback']['felt_arousal'] > 5:
                    self.features_frame[i][4] = 2 # feature['feedback']['felt_arousal']
                else:
                    self.features_frame[i][4] = 1
                self.features_frame[i][5] = feature['feedback']['felt_valence']
                i += 1
        # convert to pytorch tensor
        self.features_frame = torch.from_numpy(self.features_frame)
        #self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.features_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        #img_name = os.path.join(self.root_dir,
        #                        self.features_frame.iloc[idx, 0])
        #image = io.imread(img_name)
        feature_set = self.features_frame[idx][0:4]
        arousal = self.features_frame[idx][4]
        valence = self.features_frame[idx][5]
        #landmarks = np.array([landmarks])
        #landmarks = landmarks.astype('float').reshape(-1, 2)
        #print(torch.tensor([arousal]))
        sample = {'features': feature_set, 'arousal':torch.tensor([arousal]), 'valence': valence}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [5]:
face_dataset = FeaturesDataset(dataFile)

In [6]:
features_frame = io.loadmat(dataFile)#pd.read_csv(csv_file)
features_frame = np.zeros(shape=(1280, 4))
arousal = np.zeros(shape=(1280))
i = 0
#print(dataHeaders.shape)
#for line in dataHeaders:
subjects = dataHeaders['features']
# read each feature set for each video for this subject into the data array
for subject in subjects:
    for feature in subject:
        # get mean of means for EMG features
        mean_emg = np.mean(feature['EMG_feats'], axis=0)
        mean_emg = mean_emg[0]
        #print(mean_emg)
        features_frame[i][0] = mean_emg
        # set GSR mean
        #print(feature['GSR_feats'])
        features_frame[i][1] = feature['GSR_feats'][0][3]
        # set RES mean
        features_frame[i][2] = feature['RES_feats'][0][0]
        # set BVP mean
        #print("BVP")
        #print(feature['BVP_feats'])
        features_frame[i][3] = feature['BVP_feats'][0][0]
        features_frame[i]
        #print(feature['feedback']['felt_arousal'])
        #features_frame[i][4] = feature['feedback']['felt_arousal']
        #features_frame[i][5] = feature['feedback']['felt_valence']
        arousal[i] = feature['feedback']['felt_arousal']
        i += 1
print(arousal)
#features_frame = torch.from_numpy(features_frame).type(torch.FloatTensor)
#arousal = torch.from_numpy(arousal).type(torch.LongTensor)
#arousal = torch.from_numpy(arousal).type(torch.LongTensor)
#arousal = torch.nn.functional.softmax(arousal).data
#arousal = arousal
print(arousal)

[7.6  7.31 7.54 ... 7.01 7.17 4.03]
[7.6  7.31 7.54 ... 7.01 7.17 4.03]


In [7]:
#our class must extend nn.Module
class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier,self).__init__()
        #Our network consists of 3 layers. 1 input, 1 hidden and 1 output layer
        #This applies Linear transformation to input data. 
        self.fc1 = nn.Linear(4,4)
        
        #This applies linear transformation to produce output data
        self.fc2 = nn.Linear(4,1)
        
    #This must be implemented
    def forward(self,x):
        #Output of the first layer
        x = self.fc1(x)
        #Activation function is Relu. Feel free to experiment with this
        x = F.tanh(x)
        #This produces output
        x = self.fc2(x)
        return x
        
    #This function takes an input and predicts the class, (0 or 1)        
    def predict(self,x):
        #Apply softmax to output. 
        #pred = F.softmax(self.forward(x))
        pred = self.forward(x)#F.softmax(self.forward(x))
        ans = []
        #Pick the class with maximum weight
        for t in pred:
            if t[0]>t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [8]:
#Initialize the model        
model = MyClassifier()
#Define loss criterion
criterion = torch.nn.L1Loss()#nn.CrossEntropyLoss()
#Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
face_dataset = FeaturesDataset(dataFile)
trainloader = torch.utils.data.DataLoader(face_dataset, batch_size=64, shuffle=True)

# Define the loss
#criterion = nn.NLLLoss()
criterion = nn.L1Loss()
# Optimizers require the parameters to optimize and a learning rate
#optimizer = optim.SGD(model.parameters(), lr=0.003)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 1000
model = model.float()
i = 0
for e in range(epochs):
    running_loss = 0
    for samples in trainloader:
        #print(sample['features'])
        #print(arousal)
        #print(felt)
        # Flatten MNIST images into a 784 long vector
        #images = images.view(images.shape[0], -1)
        
        # Training pass
        #optimizer.zero_grad()
        
        #output = model(images)
        #loss = criterion(output, labels)
        #loss.backward()
        #optimizer.step()
        x = samples['features']
        x = x.float()
        y_pred = model.forward(x)
        #print(np.array(sample['arousal']))
        #print(torch.tensor([arousal]))
        #print(y_pred)
        loss = criterion(y_pred, samples['arousal'])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        i += 1
        if i % 1000 == 0:
            print(f"Training loss: {running_loss/len(trainloader)}")

/home/amy/miniconda2/envs/hw5env/lib/python3.8/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Training loss: 1.8501452033407986
Training loss: 3.0073242800310256
Training loss: 2.4250502014765516
Training loss: 4.284480552654713
Training loss: 3.7422008769586683
Training loss: 3.1457738420460375
Training loss: 4.114858020562679
Training loss: 3.770921789202839
Training loss: 2.7988731164485214
Training loss: 3.0834772860631348
Training loss: 2.592381113022566
Training loss: 3.86617088932544
Training loss: 3.74117955584079
Training loss: 2.856901769526303
Training loss: 2.2152420073281975
Training loss: 3.103222324792296
Training loss: 3.5261105411686002
Training loss: 3.899716531392187
Training loss: 2.51242680195719
Training loss: 2.326360624283552


In [10]:
face_dataset = FeaturesDataset(dataFile)
trainloader = torch.utils.data.DataLoader(face_dataset, batch_size=1, shuffle=True)

# Define the loss
#criterion = nn.NLLLoss()
criterion = nn.L1Loss()
# Optimizers require the parameters to optimize and a learning rate
#optimizer = optim.SGD(model.parameters(), lr=0.003)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 30
model = model.float()
for samples in trainloader:
    #print(sample['features'])
    #print(arousal)
    #print(felt)
    # Flatten MNIST images into a 784 long vector
    #images = images.view(images.shape[0], -1)

    # Training pass
    #optimizer.zero_grad()

    #output = model(images)
    #loss = criterion(output, labels)
    #loss.backward()
    #optimizer.step()
    x = samples['features']
    x = x.float()
    y_pred = model.forward(x)
    print("y %f x %f" %(y_pred, samples['arousal']))
    #print(np.array(sample['arousal']))
    #print(torch.tensor([arousal]))
    #print(y_pred)
    loss = criterion(y_pred, samples['arousal'])
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    #else:

y -0.926375 x 2.000000
y 3.568666 x 1.000000
y 3.558645 x 2.000000
y 4.523030 x 2.000000
y -0.966283 x 1.000000
y 0.619679 x 2.000000
y 3.518251 x 2.000000
y 1.952176 x 2.000000
y 3.233538 x 2.000000
y 0.500377 x 2.000000
y -1.025120 x 1.000000
y 4.282569 x 2.000000
y -1.044273 x 2.000000
y 4.586374 x 2.000000
y 3.435397 x 1.000000
y -1.072457 x 2.000000
y -1.081671 x 2.000000
y -1.090752 x 2.000000
y -1.099678 x 1.000000
y 4.524291 x 2.000000
y -1.117039 x 2.000000
y 2.836268 x 2.000000
y 0.120623 x 1.000000
y 2.774099 x 1.000000
y 3.323529 x 2.000000
y 4.461093 x 2.000000
y 3.299497 x 2.000000
y 4.439714 x 2.000000
y 2.616618 x 2.000000
y -1.187961 x 1.000000
y 3.249414 x 1.000000
y -0.130813 x 2.000000
y -2.370991 x 2.000000
y 4.373728 x 2.000000
y -1.223038 x 2.000000
y -1.229672 x 1.000000
y 2.358685 x 2.000000
y -1.242461 x 1.000000
y 4.316873 x 2.000000
y -0.346381 x 2.000000
y 3.112995 x 1.000000
y -2.413444 x 2.000000
y -1.271690 x 2.000000
y -1.277072 x 2.000000
y -1.282258 x

In [75]:
#Number of epochs
epochs = 10000
#List to store losses
losses = []
for i in range(epochs):
    #Precit the output for Given input
    y_pred = model.forward(features_frame)
    print("y pred")
    print(y_pred.shape)
    print(y_pred.type)
    print(y_pred)
    #Compute Cross entropy loss
    print(arousal)
    print(arousal.shape)
    loss = criterion(y_pred,arousal)
    #Add loss to the list
    losses.append(loss.item())
    #Clear the previous gradients
    optimizer.zero_grad()
    #Compute gradients
    loss.backward()
    #Adjust weights
    optimizer.step()

AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [49]:
for i in range(len(face_dataset)):
    sample = face_dataset[i]

    print(i, sample['features'].shape, sample['arousal'].shape, sample['valence'].shape)
    print(sample['features'])
    print(sample['arousal'])
    print(sample['valence'])

    if i == 3:
        plt.show()
        break

0 (4,) () ()
[  11.70429484 1716.50352589  441.8729483  -203.73998585]
7.6
7.71
1 (4,) () ()
[    9.35744446 -4327.18509003   990.31802215  4470.75750534]
7.31
8.1
2 (4,) () ()
[ 6.28137654e+00 -4.37382019e+04  5.96898714e+02  1.07425946e+04]
7.54
8.58
3 (4,) () ()
[  113.41911319  9473.42197696   -31.3058334  -2754.74099166]
6.01
4.94


In [90]:
for line in dataHeaders:
    print(line)
# features is a 32 x 40 array
features = dataHeaders['features']
#for part in range(0, len(features)):
#    for trial in range(0, len(features[0])):
print(features[2][1])
print("EMG Features")
# 2d array of mean, standard deviation
print(features[2][1]['EMG_feats'])
print("GSR Features")
#array([[array(['nbPeaks'], dtype='<U7'),
#        array(['ampPeaks'], dtype='<U8'),
#        array(['riseTime'], dtype='<U8'),
#        array(['meanGSR'], dtype='<U7'), array(['stdGSR'], dtype='<U6')]
print(features[2][1]['GSR_feats'])
print("RES Features")
#array([[array(['mean_'], dtype='<U5'), array(['std_'], dtype='<U4'),
#        array(['kurtosis_'], dtype='<U9'),
#        array(['skewness_'], dtype='<U9'),
#        array(['sp0001'], dtype='<U6'), array(['sp0102'], dtype='<U6'),
#        array(['sp0203'], dtype='<U6'), array(['sp0304'], dtype='<U6'),
#        array(['sp0407'], dtype='<U6'), array(['sp0710'], dtype='<U6'),
#        array(['sp1025'], dtype='<U6'),
#        array(['main_freq'], dtype='<U9')]]
print(features[2][1]['RES_feats'])
print("BVP Features")
#array([[array(['mean_'], dtype='<U5'), array(['HRV'], dtype='<U3'),
#        array(['meanIBI'], dtype='<U7'), array(['MSE'], dtype='<U3'),
#        array(['sp0001'], dtype='<U6'), array(['sp0102'], dtype='<U6'),
#        array(['sp0203'], dtype='<U6'), array(['sp0304'], dtype='<U6'),
#        array(['sp_energyRatio'], dtype='<U14'),
#        array(['tachogram_LF'], dtype='<U12'),
#        array(['tachogram_MF'], dtype='<U12'),
#        array(['tachogram_HF'], dtype='<U12'),
#        array(['tachogram_energy_ratio'], dtype='<U22')]]
print(features[2][1]['BVP_feats'])
#images = torch.from_numpy(dataHeaders['features'])

__header__
__version__
__globals__
features
(array([[-1.06372753e+02,  3.97953002e+02],
       [ 7.40471370e+01,  2.57770678e+02],
       [ 2.05306513e+00,  5.98757168e+00],
       [ 3.46160245e-01, -1.45525044e+00],
       [ 5.39392752e+00,  6.15447866e+00]]), array([[array(['mean_'], dtype='<U5'), array(['std_'], dtype='<U4'),
        array(['kurtosis_'], dtype='<U9'),
        array(['skewness_'], dtype='<U9'),
        array(['EMG_power'], dtype='<U9')]], dtype=object), array([[ 0.54987155,  2.41224008,  0.03270986, -0.82983959,  4.58966793,
         2.51024318,  5.22089147,  2.8303609 ,  4.18048402,  2.38668605,
         4.6310267 ,  4.52803443,  3.4129617 ,  0.33243473, -0.67574304,
         3.22879395,  5.81654855, -0.2086525 ,  3.3514443 ,  3.44921797,
         4.03485444,  4.34970661,  3.26744249,  1.94753859,  2.11219193,
         3.75166585,  2.44340265,  3.38248406,  2.63312473,  1.88812982,
         4.57138965, -0.02706612],
       [ 5.67527462,  7.11436861,  5.02645715,  5.

In [51]:
for line in dataFile:
    print(line)
print(dataFile['evals_self_arousal_periph'])
images = torch.from_numpy(dataFile['evals_self_arousal_periph'])

__header__
__version__
__globals__
evals_self_arousal_periph
evals_self_valence_periph
[[array([[(array([[0.525]]), array([[0.42105263, 0.61904762]]), array([[0.5       , 0.54166667]]), array([[0.45714286, 0.57777778]]))]],
      dtype=[('classification_rate', 'O'), ('prec', 'O'), ('recall', 'O'), ('f1s', 'O')])
  array([[(array([[0.475]]), array([[0.35294118, 0.56521739]]), array([[0.375     , 0.54166667]]), array([[0.36363636, 0.55319149]]))]],
      dtype=[('classification_rate', 'O'), ('prec', 'O'), ('recall', 'O'), ('f1s', 'O')])
  array([[(array([[0.575]]), array([[0.77777778, 0.15384615]]), array([[0.65625, 0.25   ]]), array([[0.71186441, 0.19047619]]))]],
      dtype=[('classification_rate', 'O'), ('prec', 'O'), ('recall', 'O'), ('f1s', 'O')])
  array([[(array([[0.65]]), array([[0.75, 0.55]]), array([[0.625 , 0.6875]]), array([[0.68181818, 0.61111111]]))]],
      dtype=[('classification_rate', 'O'), ('prec', 'O'), ('recall', 'O'), ('f1s', 'O')])
  array([[(array([[0.375]]), arr

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.